# Clickbait Detection using Gradient Boosting & UMAP Embeddings

## Εισαγωγή
Στο παρόν notebook παρουσιάζεται η διαδικασία εκπαίδευσης και αξιολόγησης ενός **Gradient Boosting Classifier** για την αναγνώριση clickbait τίτλων.

Για την αναπαράσταση των κειμένων, χρησιμοποιούμε διανύσματα (embeddings) που έχουν μειωθεί σε **500 διαστάσεις** μέσω της τεχνικής **UMAP**. Τα δεδομένα έχουν ήδη διαχωριστεί σε **Train**, **Validation** και **Test** sets, εξασφαλίζοντας ότι δεν υπάρχει διαρροή πληροφορίας (data leakage).

## Στόχοι
1. **Φόρτωση Προ-επεξεργασμένων Δεδομένων**: Χρήση των αρχείων Parquet (`train_umap_500`, `valid_umap_500`, `test_umap_500`).
2. **Βελτιστοποίηση Υπερπαραμέτρων (Hyperparameter Tuning)**: Χρήση της βιβλιοθήκης **Optuna** για την εύρεση των βέλτιστων παραμέτρων του μοντέλου.
3. **Καταγραφή Πειραμάτων**: Χρήση του **MLflow** για την καταγραφή των μετρικών και των μοντέλων.
4. **Τελική Αξιολόγηση**: Εκπαίδευση του βέλτιστου μοντέλου (Champion Model) στο σύνολο των δεδομένων εκπαίδευσης (Train + Valid) και αξιολόγηση στο Test set.

In [1]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# --- ΡΥΘΜΙΣΕΙΣ PATHS ---
# Προσθήκη του φακέλου Models στο path για να φορτώσουμε το mlflow_helper
# Υποθέτουμε ότι το notebook τρέχει στον φάκελο: Models/Random Forest - Gradient Boosting/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

# Ορισμός του Project Root (3 επίπεδα πάνω από τον φάκελο του script)
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))

# Ορισμός του φακέλου με τα δεδομένα (Data/Clean/UMAP)
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"📂 Project Root: {project_root}")
print(f"📂 Data Folder: {DATA_FOLDER}")

📂 Project Root: /Users/nikosgatos/PycharmProjects/Clickbait_Machine_Learning_Project
📂 Data Folder: /Users/nikosgatos/PycharmProjects/Clickbait_Machine_Learning_Project/data/clean/umap


## Φόρτωση Δεδομένων (Data Loading)

Φορτώνουμε τα αρχεία `.parquet` που περιέχουν τα embeddings (500 διαστάσεις) και τα labels.
* **Train Set**: Χρησιμοποιείται για την εκπαίδευση των υποψήφιων μοντέλων στο Optuna.
* **Validation Set**: Χρησιμοποιείται από το Optuna για την αξιολόγηση κάθε trial και την επιλογή των βέλτιστων παραμέτρων.
* **Test Set**: Κρατείται "κλειστό" και χρησιμοποιείται **μόνο** στο τέλος για την αξιολόγηση του τελικού μοντέλου.

In [2]:
def load_split_data(data_path):
    """
    Φορτώνει τα αρχεία Train/Valid/Test Parquet.
    Αναγνωρίζει αυτόματα τα UMAP features και τη στήλη 'labels'.
    """
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    # Προσθέσαμε το 'labels' που βρήκαμε στο debug
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"⏳ Έναρξη φόρτωσης δεδομένων από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"❌ Το αρχείο {filename} δεν βρέθηκε.")

        # 1. Φόρτωση DataFrame
        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 2. Εντοπισμός Features (X)
        feature_cols = [c for c in df.columns if c.startswith("umap_")]
        if not feature_cols:
            # Αν για κάποιο λόγο δεν βρεθούν, παίρνουμε όλες εκτός από τα πιθανά labels
            feature_cols = [c for c in df.columns if c not in possible_label_cols]

        # 3. Εντοπισμός Labels (y)
        label_col = None
        for col in possible_label_cols:
            if col in df.columns:
                label_col = col
                break

        # Fallback: Αν δεν βρέθηκε όνομα από τη λίστα, ψάχνουμε οποιαδήποτε στήλη δεν είναι feature
        if label_col is None:
            remaining_cols = [c for c in df.columns if c not in feature_cols]
            if len(remaining_cols) == 1:
                label_col = remaining_cols[0]
                print(f"   ⚠️ {name}: Αυτόματος εντοπισμός label στήλης: '{label_col}'")

        if label_col:
            # Αφαίρεση του label από τα features αν είχε μπει κατά λάθος
            if label_col in feature_cols:
                feature_cols.remove(label_col)

            y = df[label_col].values.astype(int)
        else:
            raise ValueError(f"⛔ Σφάλμα στο {name}: Δεν βρέθηκε στήλη label (Δοκιμάστηκαν: {possible_label_cols})")

        # 4. Μετατροπή σε Numpy Arrays
        X = df[feature_cols].values.astype(np.float32)

        # Validation
        if len(X) != len(y):
             raise ValueError(f"❌ Ασυμφωνία διαστάσεων στο {name}: X={len(X)}, y={len(y)}")

        loaded_data[name] = (X, y)
        print(f"   ✅ {name} loaded: X={X.shape}, y={y.shape} (Label col: '{label_col}')")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

⏳ Έναρξη φόρτωσης δεδομένων από: /Users/nikosgatos/PycharmProjects/Clickbait_Machine_Learning_Project/data/clean/umap
   ✅ Train loaded: X=(84835, 500), y=(84835,) (Label col: 'labels')
   ✅ Valid loaded: X=(10604, 500), y=(10604,) (Label col: 'labels')
   ✅ Test loaded: X=(10605, 500), y=(10605,) (Label col: 'labels')


##  Βελτιστοποίηση με Optuna (Hyperparameter Tuning)

Ορίζουμε την `objective` function, η οποία θα εκτελεστεί πολλές φορές (trials) από το Optuna. Σε κάθε trial:
1. Το Optuna επιλέγει έναν συνδυασμό υπερπαραμέτρων (π.χ. `learning_rate`, `n_estimators`, `max_depth`).
2. Εκπαιδεύουμε έναν `GradientBoostingClassifier` με αυτές τις παραμέτρους στο **Train Set**.
3. Υπολογίζουμε το **F1 Score** στο **Validation Set**.
4. Καταγράφουμε τα αποτελέσματα στο **MLflow**.

Σκοπός είναι να μεγιστοποιήσουμε το F1 Score στο Validation Set.

In [5]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # --- Search Space (Εύρος Αναζήτησης) ---
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "min_samples_split": trial.suggest_int("min_samples_split", 5, 20),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "random_state": 42
    }

    # Δημιουργία και Εκπαίδευση Μοντέλου
    model = GradientBoostingClassifier(**params)
    model.fit(X_tr, y_tr)

    # Πρόβλεψη στο Validation Set
    preds = model.predict(X_v)

    # Υπολογισμός Μετρικών
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    # Καταγραφή του trial στο MLflow (Nested Run)
    # Προσοχή: Εδώ καλούμε το helper αλλά περνάμε metrics ως dict
    metrics = {"val_f1": f1, "val_accuracy": acc}
    mlflow_helper.log_optuna_trial(trial, params, metrics, model, "gb_model_trial")

    return f1

## Εκτέλεση Πειράματος & Επιλογή Champion Model

Ξεκινάμε το πείραμα στο MLflow με όνομα `Clickbait_GradientBoosting_UMAP_500`.

**Βήματα:**
1. **Optuna Search**: Τρέχουμε 15 trials για να βρούμε τις καλύτερες παραμέτρους.
2. **Champion Training**: Παίρνουμε τις "νικήτριες" παραμέτρους και εκπαιδεύουμε ένα νέο μοντέλο στο **συνδυασμένο σύνολο** (Train + Validation). Αυτό γίνεται για να εκμεταλλευτούμε όλα τα διαθέσιμα δεδομένα πριν το τελικό test.
3. **Final Evaluation**: Αξιολογούμε το Champion Model στο **Test Set**.

In [6]:
EXPERIMENT_NAME = "Clickbait_GradientBoosting_UMAP_500"

# Ρύθμιση MLflow (το helper βρίσκει αυτόματα το φάκελο mlruns στο root)
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\n🚀 Έναρξη Πειράματος: {EXPERIMENT_NAME}")

with mlflow.start_run(run_name="GB_UMAP_Optimization") as run:
    # Καταγραφή γενικών παραμέτρων του πειράματος
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("model_type", "GradientBoosting")

    # --- 1. OPTUNA OPTIMIZATION ---
    print("🔎 Ξεκινά η αναζήτηση υπερπαραμέτρων...")
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler)

    # Τρέχουμε το Optimization
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=15)

    print(f"\n🏆 Best Params: {study.best_params}")
    print(f"🏆 Best Val F1: {study.best_value:.4f}")

    # --- 2. CHAMPION MODEL TRAINING (TRAIN + VAL) ---
    print("\n⚙️ Εκπαίδευση του Champion Model (Train + Valid)...")

    best_params = study.best_params
    best_params["random_state"] = 42 # Διασφάλιση επαναληψιμότητας

    final_model = GradientBoostingClassifier(**best_params)

    # Ένωση Train και Validation sets
    X_full_train = np.concatenate((X_train, X_val))
    y_full_train = np.concatenate((y_train, y_val))

    final_model.fit(X_full_train, y_full_train)

    # Log του τελικού μοντέλου στο MLflow
    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # --- 3. FINAL EVALUATION (TEST SET) ---
    print("\n📈 Αξιολόγηση στο Test Set...")

    # Χρήση της βοηθητικής συνάρτησης για plots (ROC, Confusion Matrix) και metrics
    test_f1 = mlflow_helper.evaluate_and_log_metrics(final_model, X_test, y_test, prefix="test")

    print(f"\n✅ Το πείραμα ολοκληρώθηκε επιτυχώς! (Run ID: {run.info.run_id})")

/Users/nikosgatos/PycharmProjects/Clickbait_Machine_Learning_Project/.venv/lib/python3.12/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2025/12/24 17:22:41 INFO mlflow.tracking.fluent: Experiment with name 'Clickbait_GradientBoosting_UMAP_500' does not exist. Creating a new experiment.
[I 2025-12-24 17:22:41,196] A new study created in memory with name: no-name-be5c5b83-865e-458b-ae1d-2b798859ea42


🚀 MLflow tracking URI set to: /Users/nikosgatos/PycharmProjects/Clickbait_Machine_Learning_Project/mlruns
🚀 MLflow experiment set to: Clickbait_GradientBoosting_UMAP_500

🚀 Έναρξη Πειράματος: Clickbait_GradientBoosting_UMAP_500
🔎 Ξεκινά η αναζήτηση υπερπαραμέτρων...


[W 2025-12-24 17:23:19,974] Trial 0 failed with parameters: {'n_estimators': 175, 'learning_rate': 0.17254716573280354, 'max_depth': 7, 'min_samples_split': 14, 'subsample': 0.7468055921327309} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/nikosgatos/PycharmProjects/Clickbait_Machine_Learning_Project/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/gt/16rl7qln7xbbhjw7wqx2xvkr0000gn/T/ipykernel_18126/950005772.py", line 19, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=15)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/gt/16rl7qln7xbbhjw7wqx2xvkr0000gn/T/ipykernel_18126/1782700106.py", line 14, in objective
    model.fit(X_tr, y_tr)
  File "/Users/nikosgatos/PycharmProjects/Clickbait_Machine_Le

KeyboardInterrupt: 

## Συμπεράσματα

Η διαδικασία ολοκληρώθηκε. Το τελικό μοντέλο εκπαιδεύτηκε με τις βέλτιστες παραμέτρους και αξιολογήθηκε σε άγνωστα δεδομένα (Test Set).

**Επόμενα Βήματα:**
* Έλεγχος των γραφημάτων (Confusion Matrix, ROC Curve) στο MLflow UI.
* Σύγκριση των αποτελεσμάτων με άλλα μοντέλα (π.χ. SGD, Random Forest) για την επιλογή της τελικής λύσης.